# Optimized Checkpointing with Tensorstore

Orbax relies on [Tensorstore](https://google.github.io/tensorstore/) to store
individual arrays in a checkpoint. Tensorstore provides efficient, scalable library for reading and writing arrays.

Until recently, however, our use of Tensorstore came with a few drawbacks. Chief among them was the fact that every parameter in a training state would be saved as a separate directory. This approach can be quite performant, even for models with hundreds of billions of parameters, *provided that model layers are stacked*. Otherwise, hundreds or thousands of directories may be created in the checkpoint.

This fact can lead to very slow restore times, which is undesirable in and of itself, but is particularly painful for jobs that may be preempted frequently and need to restart, for example.

While it is slightly less of a concern at save time, since writes to disk can happen asynchronously, the synchronous portion of the save can still be slow as many directories are created.

Additionally, if individual parameters are small, storage may be wasted on filesystems with minimum file sizes.

## Introducing OCDBT

The new, optimized checkpoint format provided by Orbax is backed by Tensorstore's [OCDBT](https://google.github.io/tensorstore/kvstore/ocdbt/index.html) driver (optionally-cooperative distributed B-tree).

For practical purposes, this means that we will no longer store one parameter per directory, but will aggregate many parameters into a smaller set of large files.

Empirically, we have observed substantial speed-ups in both save and restore when using OCDBT.

### Save Performance (sec)

<img src=https://orbax.readthedocs.io/en/latest/_static/checkpoint_benchmarks_save_ocdbt.png>

### Restore Performance (sec)

<img src=https://orbax.readthedocs.io/en/latest/_static/checkpoint_benchmarks_restore_ocdbt.png>

## Checkpoint Format

Concretely, what does the new checkpoint format look like in comparison to the old?

### Old Format

In [ ]:
f = """
path/to/my/checkpoint/dir/
  0/
    state/
      layer0.param0/
        .zarray
        0.0
        0.1
        1.0
        1.1
      layer1.param0/
        .zarray
        0.0
      ...
    <another_item>/
      ...
  1/
    ...
  2/
    ...

Note: in this case, `0.0`, `0.1`, etc. provides an indication of how the array
was sharded when originally saved.
"""

### New Format

In [ ]:
f = """
path/to/my/checkpoint/dir/
  0/
    state/
      checkpoint  # legacy msgpack file, stores tree structure
      tree_metadata  # (maybe) new proto file, stores tree structure
      d/  # array data stored here
        012b2c6e5c9d2a16c240a59d5f0f35c0
        056e0816bdc5496a86251e58a0ec202b
        ...
      manifest.0000000000000001
      ...
      manifest.ocdbt
    <another_item>/
      ...
  1/
    ...
  2/
    ...
"""

## Enabling OCDBT

In [ ]:
import jax
import tempfile
import subprocess
import os
from etils import epath

import orbax.checkpoint as ocp

In [ ]:
# Initialize PyTreeCheckpointHandler with `use_ocdbt=True`.
# This option already defaults to True, so it's optional to pass it in.
ckptr = ocp.Checkpointer(ocp.PyTreeCheckpointHandler(use_ocdbt=True))

## Additional Notes

All checkpoints previously produced by Orbax in the old format will still be
readable when OCDBT is enabled. However, if a checkpoint is produced in the OCDBT format, it cannot be read if the OCDBT feature is disabled.

## Custom Chunk Sizes
Orbax [Zarr3](https://zarr-specs.readthedocs.io/en/latest/specs.html#specifications), a multidimensional array storage format, offers customizable chunk sizes in bytes for optimal memory management.  The default chunk size, which corresponds one-to-one with the array shard size, can cause out-of-memory errors when reading on hosts with different sharding layouts.  For example, this can often arise when arrays are saved with a fully-sharded sharding, but loaded with a fully-replicated sharding.  To prevent this, set `chunk_byte_size` smaller than or equal to the anticipated read size.  Anything above 1MB generally won't affect impact performance. Consider following example:

In [ ]:
# setup checkpoint data
array_len = 8 * 1024
key = jax.random.PRNGKey(0)
key, subkey = jax.random.split(key)
pytree = {
          'a': jax.random.normal(subkey, (array_len, ), dtype=jax.numpy.float32), # 32KB
          'b': jax.random.normal(subkey, (array_len * 2, ), dtype=jax.numpy.float32), # 64KB
}

# create save_args to customize the chunk_byte_size
save_args = jax.tree_util.tree_map(
    lambda x: ocp.SaveArgs(
        chunk_byte_size=
        1024,  # 1KB
    ),
    pytree,
)

In [ ]:
temp_dir = tempfile.TemporaryDirectory()
mgr = ocp.CheckpointManager(epath.Path(temp_dir.name),
                            item_handlers=ocp.PyTreeCheckpointHandler(use_zarr3=True)) # make sure zarr3 is enabled

mgr.save(
  0,
  args=ocp.args.PyTreeSave(
      pytree,
      save_args=save_args,
  ),
)

mgr.close()

## Custom OCDBT Data File Size
To improve file I/O parallelism when working with large files on remote storages like GCS, use the `PyTreeSaveArgs.ocdbt_target_data_file_size` parameter to control the size of output files.

### BEFORE

In [ ]:
def print_directory_file_size(dir: epath.Path) -> None:
  print(f"dir={dir}:")
  for f in data_dir.iterdir():
    if f.is_file():
      print(f"file={f.name}, size={f.stat().length}")

# continue from above example, examine the data file sizes
data_dir = epath.Path(temp_dir.name) / '0'/ 'default'/ 'ocdbt.process_0'/ 'd'
print_directory_file_size(data_dir)

### AFTER

In [ ]:
temp_dir = tempfile.TemporaryDirectory()
mgr = ocp.CheckpointManager(temp_dir.name,
                            item_handlers=ocp.PyTreeCheckpointHandler(use_zarr3=True))

mgr.save(
  0,
  args=ocp.args.PyTreeSave(
      pytree,
      save_args=save_args,
      ocdbt_target_data_file_size=10 * 1024, #10 KB, should be much larger than chunk_byte_size
  ),
)

mgr.close()

data_dir = epath.Path(temp_dir.name) / '0'/ 'default'/ 'ocdbt.process_0'/ 'd'
print_directory_file_size(data_dir)